In [1]:
from pyspark.sql import SparkSession

# إنشاء SparkSession
spark = SparkSession.builder \
    .appName("NullHandlingExample") \
    .getOrCreate()
from pyspark.sql import SparkSession

# إنشاء SparkSession
spark = SparkSession.builder \
    .appName("NullHandlingExample") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/29 00:30:16 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/10/29 00:30:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/29 00:30:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# هنا بنريد  الاستريم من الجهاز عن طريق السوكيت  
# ونقدر نعمل كده لو كتبا في التريمينل كده
# nc  -l 1412                 -----1412  are port  numberr

df=spark.readStream.format('socket')\
    .option('host','localhost')\
        .option('port',1412).load()
df

25/10/29 00:32:02 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


DataFrame[value: string]

In [ ]:
# هنا  بنعمل رايت ل الدااتا وبنعمل تريجر ل عشان يروح يششوف الداتاكل 5 ثوواني 
writer = df.writeStream.outputMode("append").format("console").trigger(processingTime="5 seconds")
#query.awaitTermination()
query= writer.start()

25/10/29 00:26:04 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
25/10/29 00:26:04 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-028c24ff-3004-4840-888a-109e3abcdc14. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/29 00:26:04 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [21]:
# لو هنوقف الاستريم 
#query.stop()

socket هنعمل مثال كامل مع  
---------------
الهدف من الموضوع اني استقبل استريم جي من اللوكل من البورت الي انا حدتته الي هو 1412
وبعدين هقسم كل رساله تجيلي من الاستريم ل كلامات واعدها 
وبعدين اعملها جرب باي بحيث يظهر كل جمله جات وجمبها عدد الكلمات الي فيها
وبعدين اعمل رايت بقا واعرض الداتا فريم دي في الكونسل 


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SocketStreamExample") \
    .getOrCreate()


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/10/30 03:46:07 WARN Utils: Your hostname, ahmed-refat-VirtualBox, resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
25/10/30 03:46:07 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/10/30 03:46:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [ ]:
#“استقبل الداتا اللي جاية على الجهاز المحلي (localhost) من البورت رقم 1412”.
df = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 1412) \
    .load()


25/10/30 03:47:00 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [4]:
# هنبدء نقسم البيانات الي جايه ل كلمات 
# explode :- put every word in row
# split :- split every sentacs to words 
import pyspark.sql.functions as fn

words = df.select(fn.explode(fn.split(df.value, " ").alias("word")))

In [ ]:
# groub by
from pyspark.sql.functions import count

words_count =words .groupBy('word').count()

# 5️⃣ نطبع الناتج على الكونسول كل 10 ثواني
query = words_count .writeStream \
    .outputMode('complete')\
    .format('consol')\
    .trigger(processingTime='10 seconds')\
    .start()

In [ ]:
# 1️⃣ نبدأ بـ SparkSession
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split

spark = SparkSession.builder \
    .appName("SocketStreamingExample") \
    .getOrCreate()


# 2️⃣ نقرأ البيانات من socket
# تأكد إنك مشغل nc أولاً في التيرمنال التاني:
# nc -lk 1412
lines = spark.readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 1412) \
    .load()

# 3️⃣ قسم كل سطر لكلمات منفصلة
words = lines.select(explode(split(lines.value, " ")).alias("word"))

# هنا بنسمي العمود الجديد "word"
# لو نسيت الـ alias("word") هيتسمى col وهتظهر نفس الغلطة اللي جاتلك

# 4️⃣ نحسب عدد مرات ظهور كل كلمة
from pyspark.sql.functions import count

words_count = words.groupBy("word").count()

# 5️⃣ نطبع الناتج على الكونسول كل 10 ثواني
query = words_count.writeStream \
    .outputMode("complete") \
    .format("console") \
    .trigger(processingTime="10 seconds") \
    .start()



-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|    1412|    1|
|   refat|    2|
|     you|    1|
|     how|    1|
|   hello|    1|
|   ahmed|    2|
|    gggh|    1|
|      is|    1|
|      ok|    1|
|    aggg|    1|
|     are|    1|
|straming|    1|
|        |    4|
|    this|    1|
| message|    1|
+--------+-----+



In [15]:
query .stop()